## Importing Required Libraries

In [1]:
import numpy as np
import os

## Loading the files

In [2]:
#Paths for blur
path_to_X_blur_train = './data/X/train'
path_to_X_blur_test = './data/X/test'
path_to_Y_blur_train= './data/Y/train'
path_to_Y_blur_test = './data/Y/test'

In [19]:
#Paths for noise
path_to_X_noise_train = './data_noise/X/train'
path_to_X_noise_test =  './data_noise/X/test'
path_to_Y_noise_train= './data_noise/Y/train'
path_to_Y_noise_test = './data_noise/Y/test'

In [29]:
#Paths for scratched
path_to_X_scratch_train = './data_scratch/X/train'
path_to_X_scratch_test = './data_scratch/X/test'
path_to_Y_scratch_train= './data_scratch/Y/train'
path_to_Y_scratch_test = './data_scratch/Y/test'

In [38]:
#Path for water_spilled
path_to_X_water_train = './data_water_spilled/X/train'
path_to_X_water_test = './data_water_spilled/X/test'
path_to_Y_water_train = './data_water_spilled/Y/train'
path_to_Y_water_test= './data_water_spilled/Y/test'

In [3]:
## Writing a loading function
def loader(paths,loaded):
    for image in os.scandir(paths):
        loaded.append(np.load(image))

In [4]:
#Loading Blur Data
x_train_blur,x_test_blur,y_train_blur,y_test_blur =[],[],[],[]
loader(path_to_X_blur_train,x_train_blur)
loader(path_to_Y_blur_train,y_train_blur)
loader(path_to_X_blur_test,x_test_blur) 
loader(path_to_Y_blur_test,y_test_blur)
x_train_blur = np.stack(x_train_blur,axis=0)
y_train_blur = np.stack(y_train_blur,axis=0)

In [12]:
x_test_blur = np.stack(x_test_blur,axis=0)
y_test_blur = np.stack(y_test_blur,axis=0)

In [20]:
#Loading Noise Data
x_train_noise,x_test_noise,y_train_noise,y_test_noise =[],[],[],[]
loader(path_to_X_noise_train,x_train_noise)
loader(path_to_Y_noise_train,y_train_noise)
loader(path_to_X_noise_test,x_test_noise) 
loader(path_to_Y_noise_test,y_test_noise)
x_train_noise = np.stack(x_train_noise,axis=0)
y_train_noise = np.stack(y_train_noise,axis=0)

In [23]:
x_test_noise = np.stack(x_test_noise,axis=0)
y_test_noise = np.stack(y_test_noise,axis=0)

In [30]:
#Loading the Strach Data
x_train_scratch,x_test_scratch,y_train_scratch,y_test_scratch =[],[],[],[]
loader(path_to_X_scratch_train,x_train_scratch)
loader(path_to_Y_scratch_train,y_train_scratch)
loader(path_to_X_scratch_test,x_test_scratch) 
loader(path_to_Y_scratch_test,y_test_scratch)
x_train_scratch = np.stack(x_train_scratch)
y_train_scratch = np.stack(y_train_scratch)

In [34]:
x_test_scratch = np.stack(x_test_scratch)
y_test_scratch = np.stack(y_test_scratch)

In [39]:
#Loading
x_train_water,x_test_water,y_train_water,y_test_water =[],[],[],[]
loader(path_to_X_water_train,x_train_water)
loader(path_to_Y_water_train,y_train_water)
loader(path_to_X_water_test,x_test_water) 
loader(path_to_Y_water_test,y_test_water)
x_train_water = np.stack(x_train_water)
y_train_water = np.stack(y_train_water)

In [42]:
x_test_water = np.stack(x_test_water)
y_test_water = np.stack(y_test_water)

## Importing required libraries for building CNN Models

## Building CNN models for each of the possible damages to photos

In [5]:
## Importing TensorFlow
import tensorflow as tf

In [6]:
## Importing Required layers for creation of CNN from TensofFlow.Keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Concatenate, UpSampling2D

In [7]:
## Importing the base Model from TensorFlow.Keras
from tensorflow.keras.models import Model

### Utilizing Sequential Learning to train the model to deal with different types of Image deteriorations

In [8]:
## Creating a CNN model that can be used for all our datasets, i.e, to resolve Blurring,Scratches,Noise and Water-Spills
def model_image_restore(input_shape):
    inputs = Input(input_shape)
    
    #Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    p2 = MaxPooling2D((2, 2))(c2)

    # Bottleneck
    b = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)

    # Decoder
    u1 = UpSampling2D((2, 2))(b)
    concat1 = Concatenate()([u1, c2])
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(concat1)

    u2 = UpSampling2D((2, 2))(c3)
    concat2 = Concatenate()([u2, c1])
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat2)

    # Output layer
    outputs = Conv2D(3, (1, 1), activation='sigmoid')(c4)

    return tf.keras.Model(inputs=inputs, outputs=outputs)

## Model Compilation 

In [9]:
## Creating the CNN for restoring the images in our dataset by using our model_image_restore
input_shape = (256,256,3) # the shape that every single input image has to be in to be fed to the CNN
model_image_restore = model_image_restore(input_shape)

In [10]:
## Compiling the model_unbur
model_image_restore.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

## Sequential Step 1 - Train the Model to Unblur the Images

In [11]:
## Training the model_image_restore using our pre-processed data
Training_unblur = model_image_restore.fit(x_train_blur,y_train_blur,epochs=15,batch_size=32,validation_split=0.2) 

Epoch 1/15
19/19 [==============================] - 376s 20s/step - loss: 0.0680 - accuracy: 0.6521 - val_loss: 0.0283 - val_accuracy: 0.7961
Epoch 2/15
19/19 [==============================] - 371s 20s/step - loss: 0.0171 - accuracy: 0.7633 - val_loss: 0.0142 - val_accuracy: 0.7842
Epoch 3/15
19/19 [==============================] - 370s 20s/step - loss: 0.0104 - accuracy: 0.7723 - val_loss: 0.0094 - val_accuracy: 0.8111
Epoch 4/15
19/19 [==============================] - 374s 20s/step - loss: 0.0079 - accuracy: 0.7922 - val_loss: 0.0077 - val_accuracy: 0.8201
Epoch 5/15
19/19 [==============================] - 371s 20s/step - loss: 0.0068 - accuracy: 0.8195 - val_loss: 0.0069 - val_accuracy: 0.8440
Epoch 6/15
19/19 [==============================] - 373s 20s/step - loss: 0.0068 - accuracy: 0.8286 - val_loss: 0.0067 - val_accuracy: 0.8482
Epoch 7/15
19/19 [==============================] - 372s 20s/step - loss: 0.0061 - accuracy: 0.8376 - val_loss: 0.0064 - val_accuracy: 0.8497
Epoch 

In [14]:
## Testing the model_image_restore for blurry images
unblur_outputs = model_image_restore.predict(x_test_blur)

8/8 [==============================] - 36s 5s/step


In [16]:
## To Find the loss and accuraccy of the predictions made by model_image_restore
loss,accuracy_unblur = model_image_restore.evaluate(x_test_blur,y_test_blur)
print(f"Test Accuraccy For Unblurring The Images: {accuracy_unblur * 100:.2f}%")

8/8 [==============================] - 35s 4s/step - loss: 0.0054 - accuracy: 0.9159
Test Accuraccy For Unblurring The Images: 91.59%


## Sequential Step 2 - Train the Model to De-Noise the Images

In [22]:
## Training the model_image_restore using our pre-processed noisy data
Training_denoise = model_image_restore.fit(x_train_noise,y_train_noise,epochs=15,batch_size=32,validation_split=0.2)

Epoch 1/15
18/18 [==============================] - 426s 24s/step - loss: 0.0026 - accuracy: 0.7871 - val_loss: 0.0018 - val_accuracy: 0.7402
Epoch 2/15
18/18 [==============================] - 441s 24s/step - loss: 0.0017 - accuracy: 0.7829 - val_loss: 0.0013 - val_accuracy: 0.7113
Epoch 3/15
18/18 [==============================] - 422s 23s/step - loss: 0.0013 - accuracy: 0.8253 - val_loss: 0.0011 - val_accuracy: 0.8184
Epoch 4/15
18/18 [==============================] - 353s 20s/step - loss: 0.0011 - accuracy: 0.8217 - val_loss: 9.0953e-04 - val_accuracy: 0.8102
Epoch 5/15
18/18 [==============================] - 356s 20s/step - loss: 9.1059e-04 - accuracy: 0.8066 - val_loss: 8.3627e-04 - val_accuracy: 0.8131
Epoch 6/15
18/18 [==============================] - 356s 20s/step - loss: 9.4494e-04 - accuracy: 0.8068 - val_loss: 7.4394e-04 - val_accuracy: 0.7974
Epoch 7/15
18/18 [==============================] - 356s 20s/step - loss: 8.8345e-04 - accuracy: 0.8070 - val_loss: 7.8155e-04 -

In [24]:
## Testing the model_image_restore
denoise_outputs = model_image_restore.predict(x_test_noise)

8/8 [==============================] - 36s 4s/step


In [27]:
## To Find the loss and accuraccy of the predictions made by model_image_restore
loss,accuracy_denoise = model_image_restore.evaluate(x_test_noise,y_test_noise)
print(f"Test Accuraccy For De-Noising The Images: {accuracy_denoise * 100:.2f}%")

8/8 [==============================] - 36s 4s/step - loss: 0.0421 - accuracy: 0.1680
Test Accuraccy For De-Noising The Images: 16.80%


## Sequential Step 3 -Train the Model to Repair the Scratches on the Images

In [32]:
## Training the model_image_restore using our pre-processed scratched image data
Training_unscratch = model_image_restore.fit(x_train_scratch,y_train_scratch,epochs=10,batch_size=32,validation_split=0.2)

Epoch 1/10
20/20 [==============================] - 412s 20s/step - loss: 0.0252 - accuracy: 0.5267 - val_loss: 0.0088 - val_accuracy: 0.7626
Epoch 2/10
20/20 [==============================] - 394s 20s/step - loss: 0.0045 - accuracy: 0.7383 - val_loss: 0.0028 - val_accuracy: 0.7326
Epoch 3/10
20/20 [==============================] - 391s 20s/step - loss: 0.0022 - accuracy: 0.8419 - val_loss: 0.0018 - val_accuracy: 0.8764
Epoch 4/10
20/20 [==============================] - 480s 24s/step - loss: 0.0016 - accuracy: 0.8811 - val_loss: 0.0014 - val_accuracy: 0.8874
Epoch 5/10
20/20 [==============================] - 487s 24s/step - loss: 0.0014 - accuracy: 0.8885 - val_loss: 0.0013 - val_accuracy: 0.8955
Epoch 6/10
20/20 [==============================] - 418s 21s/step - loss: 0.0012 - accuracy: 0.8678 - val_loss: 0.0012 - val_accuracy: 0.8765
Epoch 7/10
20/20 [==============================] - 403s 20s/step - loss: 0.0011 - accuracy: 0.8593 - val_loss: 0.0011 - val_accuracy: 0.8821
Epoch 

In [35]:
## Testing the model_image_restore
unscratch_outputs = model_image_restore.predict(x_test_scratch)

9/9 [==============================] - 38s 4s/step


In [36]:
## To Find the loss and accuraccy of the predictions made by model_image_restore
loss,accuracy_unscratch = model_image_restore.evaluate(x_test_scratch,y_test_scratch)
print(f"Test Accuraccy For Un Scratching The Images: {accuracy_unscratch * 100:.2f}%")

9/9 [==============================] - 40s 4s/step - loss: 0.0011 - accuracy: 0.8717
Test Accuraccy For Un Scratching The Images: 87.17%


## Sequential Step 4 - Train the Model to Repair Water-Spilled Images

In [41]:
## Training the model_image_restore using our pre-processed water_spill image data
Training_despill = model_image_restore.fit(x_train_water,y_train_water,epochs=10,batch_size=32,validation_split=0.2)

Epoch 1/10
20/20 [==============================] - 404s 20s/step - loss: 0.0122 - accuracy: 0.8270 - val_loss: 0.0096 - val_accuracy: 0.8413
Epoch 2/10
20/20 [==============================] - 399s 20s/step - loss: 0.0081 - accuracy: 0.8590 - val_loss: 0.0073 - val_accuracy: 0.8412
Epoch 3/10
20/20 [==============================] - 398s 20s/step - loss: 0.0068 - accuracy: 0.8604 - val_loss: 0.0065 - val_accuracy: 0.8665
Epoch 4/10
20/20 [==============================] - 395s 20s/step - loss: 0.0058 - accuracy: 0.8699 - val_loss: 0.0060 - val_accuracy: 0.8767
Epoch 5/10
20/20 [==============================] - 394s 20s/step - loss: 0.0059 - accuracy: 0.8728 - val_loss: 0.0066 - val_accuracy: 0.8623
Epoch 6/10
20/20 [==============================] - 391s 20s/step - loss: 0.0060 - accuracy: 0.8627 - val_loss: 0.0057 - val_accuracy: 0.8620
Epoch 7/10
20/20 [==============================] - 392s 20s/step - loss: 0.0052 - accuracy: 0.8760 - val_loss: 0.0052 - val_accuracy: 0.8780
Epoch 

In [43]:
## Testing the model_image_restore
despill_outputs = model_image_restore.predict(x_test_water)

9/9 [==============================] - 39s 4s/step


In [44]:
## To Find the loss and accuraccy of the predictions made by model_image_restore
loss,accuracy_despill = model_image_restore.evaluate(x_test_water,y_test_water)
print(f"Test Accuraccy For De Spilling The Images: {accuracy_despill * 100:.2f}%")

9/9 [==============================] - 39s 4s/step - loss: 0.0048 - accuracy: 0.8507
Test Accuraccy For De Spilling The Images: 85.07%


In [45]:
## Saving the model weights
model_image_restore.save_weights('saved_weights_of_model.h5')

In [46]:
## Saving the Entire Model
model_image_restore.save('path_to_saved_model.h5')

## Data Post - Processing

In [47]:
## Creating a Size Finding Helper Function that will help post-processing by providing it with the original size of the image in question.
def SizeFinder(count,Index_Name):
    Indx = Index_Name[1][count]
    Sizes = Index_Name[0]
    return [Sizes[Indx],Indx]

In [49]:
## Converting the output images from the CNN models into their original forms, i.e, De-Normalizing the images then Resizing them.
Sample_Restored = []
def restorer(model_output,RestoredStorer,Index_Name):
    count=0 #maintaing the count to find the associated original size for resizing purposes
    for image in model_output:
        #De-Normalizing the image back from 0,1's
        de_normalized = (image*255).astype('uint8') 
        #Obtaining the Original_Sizes
        actual_size = SizeFinder(count,Index_Name) #Calling the Function that will return the original size of the image being processed
        Indx_number = actual_size[1]
        actual_width = actual_size[0][1] #Taking the actual_width of the image to pass through the resize function
        actual_height = actual_size[0][0] #Taking the actual_height of the image to pass through the resize function
        ##Resizing to Original Dimensions
        resized = cv2.resize(image,(actual_width,actual_height))
        #Storing this De-Normalized and Resized Image
        RestoredStorer.append(resized)
        count+=1 # Incrementing the count

In [50]:
#Loading Indexes
import pickle
with open('lists.pkl','rb') as f:
    Index_Blur,Index_Noise,Index_Scratch,Index_Water = pickle.load(f)

In [51]:
##passing the model_unblur_outputs through the post-processing functions
Unblurred = [] #To store the post-processed Output Images
restorer(model_unblur_outputs,Unblurred)

In [52]:
##passing the model_denoise_outputs through the post-processing functions
De_Noised = [] #To store the post-processed Output Images
restorer(model_denoise_outputs,De_Noised)

In [53]:
##passing the model_unscratch_outputs through the post-processing functions
Un_Scratch = [] #To store the post-processed Output Images
restorer(model_unscratch_outputs,Un_Scratch)

In [54]:
##passing the model_despill_outputs through the post-processing functions
De_Spill = [] #To store the post-processed Output Images
resotrer(model_despill_outputs,De_Spill)

## Saving the Restored Images in Folders

In [55]:
##Already Imported the Required Libraries at the start
## first create path to the saving folder
#writing a function to creation of save folders
def folder_creation(path_to_folder):
    if os.path.exists(path_to_folder):
        shutil.rmtree(path_to_folder)
    shutil.mkdir(path_to_folder)

In [56]:
##using the folder_creation function to create a saving folder for Unblurred photos outputter by the CNN Model
path_to_unblur = './data/Unblurred'
folder_creation(path_to_unblur)

In [57]:
##using the folder_creation function to create a saving folder for De_Noised photos outputter by the CNN Model
path_to_denoise = './data_noise/De_Noised'
folder_creation(path_to_denoise)

In [58]:
##using the folder_creation function to create a saving folder for Un_Scratched photos outputter by the CNN Model
path_to_unscratch = './data_scratch/Un_Scratched'
folder_creation(path_to_unscratch)

In [59]:
##using the folder_creation function to create a saving folder for De_Spilled photos outputter by the CNN Model
path_to_despill='./data_waterspilled/De_Spill'
folder_creation(path_to_despill)

In [60]:
#writing a function to save the restored images
def saver(outputs,path_to_save,category_name):
    count = 0 #Maintaing the count, useful in naming the files to be saved
    for image in outputs:
        img = Image.fromarray(image) #converting the np.array into an Image to be saved into a folder
        names=path_to_save.split()
        saving_name = category_name + str(count) + '.jpg' #creating the name of the file to be saved in the folder
        img.save(os.path.join(path_to_save,saving_name))
        count+=1 #Incrementing the count

In [61]:
## saving the unblurred images by passing through the saver function
saver(Unblurred,path_to_unblur,'Unblur')

In [62]:
##saving the De-Noised images by passing through the saver function
saver(De_Noised,path_to_denoise,'De_Noise')

In [63]:
##saving the Un_Scratched images by passing through the saver function
saver(Un_Scratched,path_to_unscratch,'Un_Scratch')

In [64]:
##saving the De_Spilled images by passing through the saver function
saver(De_Spill,path_to_despill,'De_Spill')